Crear un ejemplo genérico de clasificación,
crearemos un conjunto de datos sintéticos que contiene 10 millones de casos. 

In [11]:
from sklearn.datasets import make_classification
import  numpy as  np

X,y = make_classification(n_samples=10**7, n_features=5, n_informative=3, random_state=101)
D = np.c_[y,X]
#the saved file should be around 1,46 GB
np.savetxt('resources/large_dataset_10__7.csv', D, delimiter=",") 

# liberar la memoria 
del(D, X, y)

El truco para gestionar grandes volúmenes de datos sin cargar demasiados
megabytes (o gigabytes) de datos en la memoria es actualizar de forma
incremental los parámetros del algoritmo utilizando sólo una parte de los
ejemplos cada vez, esto es posible en **Scikit-learn** gracias al método *.partial_fit()*, que se ha
puesto a disposición de un cierto número de algoritmos supervisados y no
supervisados. Este método se deonoma **incremental learning** 
  
* Los trozos de datos que se introducen de forma incremental en el algoritmo de aprendizaje se
denominan lotes (batches). El tamaño de los lotes depende generalmente de la memoria disponible(cuanto mayores sean los trozos de datos, mejor, ya que la muestra de datos obtendrá más representantes de las distribuciones de
datos).
 
* Los algoritmos de aprendizaje incremental funcionan bien con datos en el intervalo de [-1,+1] o [0,+1] (por ejemplo,
Multinomial Bayes no acepta valores negativos). Sin embargo, para escalar a un rango tan preciso, es necesario conocer de antemano el rango de cada variable. Como alternativa, tiene que hacer una de estas cosas: pasar todos los
datos de una vez, registrar los valores mínimo y máximo, o derivarlos del primer lote, recortando las observaciones siguientes que superen los valores máximo y mínimo iniciales.

* La validación es un flujo de lotes, que puede conseguirse de dos maneras: -Validar de forma progresiva; es decir, probar primero cómo predice el modelo los trozos de datos recién llegados antes de pasarlos a
entrenamiento. -Reserve algunas observaciones de cada trozo. Esta
última es también la mejor forma de reservar una muestra para la
*GridSearchCV()* o para alguna otra optimización.

En nuestro ejemplo, confiamos al **SGDClassifier** una pérdida logarítmica(análoga a una regresión logística) para que aprenda a predecir un resultado binario dado 10**7 datos

In [33]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# read the big data in chunk with 10000 of length
streaming = pd.read_csv('resources/large_dataset_10__7.csv', header=None, chunksize=10000)

learner = SGDClassifier(loss='log_loss', max_iter=100)

# Utilizamos el MinMaxScaler para registrar el rango de cada variable en el primer lote.
minmax_scaler = MinMaxScaler(feature_range=(0, 1))

cumulative_accuracy = list()

for n, chunk in enumerate(streaming):
    if n == 0:
        minmax_scaler.fit(chunk.iloc[:, 1:].values)
    X = minmax_scaler.transform(chunk.iloc[:, 1:].values)
    # utilizaremos la regla de que si supera uno de los límites[0,+1], se recortan al más cercano.
    X[X > 1] = 1
    X[X < 0] = 0

    y = chunk.iloc[:, 0]

   # a partir del décimo lote, registraremos la precisión en cada lote
    if n > 8:
        cumulative_accuracy.append(learner.score(X, y))
    learner.partial_fit(X, y, classes=np.unique(y))

print('Progressive validation mean accuracy %0.3f' % np.mean(cumulative_accuracy))


Progressive validation mean accuracy 0.708


Varios algoritmos funcionan utilizando el aprendizaje incremental. Para la
clasificación:
* **sklearn.naive_bayes.BernoulliNB**
* **sklearn.naive_bayes.MultinomialNB**
* **sklearn.linear_model.Perceptron**
* **sklearn.linear_model.SGDClassifier**
* **sklearn.linear_model.PassiveAggressiveClassifier**
 
Para la regresión:
* **sklearn.linear_model.SGDRegressor**
* **sklearn.linear_model.PassiveAggressiveRegressor**
